In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache, SQLiteCache
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader

import pandas as pd

import os
import re

from dotenv import load_dotenv
load_dotenv()

def get_file_names(folder_path, format=".pdf"):
    """
    주어진 폴더 내에 있는 PDF 파일들의 이름을 리스트로 반환합니다.
    """
    import os
    
    try:
        all_files = os.listdir(folder_path)
        pdf_files = [file.replace(format,"") for file in all_files if file.lower().endswith(format)]
        
        return pdf_files
    except FileNotFoundError:
        print(f"Error: 폴더 '{folder_path}'를 찾을 수 없습니다.")
        return []
    except Exception as e:
        print(f"Error: {e}")
        return []
    

In [2]:
from loader import load_process_split_doc, load_process_split_doc_law

# tax_etc 파일들 정의의
doc_options = [
    {
        "filename":"연말정산_신고안내", 
        "page_ranges": [(17, 426)],
        "table_pages_range": "17-425",
        "replace_string": [
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"},
            {"target_str": r"01. 2024년 귀속 연말정산 개정세법 요약\n\d+|"  },
            {"target_str": r"II. 2024년 귀속 연말정산 주요 일정\n\d+|"},
            {"target_str": r"III\. 원천징수의무자의 연말정산 중점 확인사항\n\d+|"},
            {"target_str": r"원천징수의무자를 위한 \n2024년 연말정산 신고안내\n\d+|"},
            {"target_str": r"Ⅰ\. 근로소득\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수 및 연말정산\n\d+|"},
            {"target_str": r"III\. 근로소득공제, 인적공제, 연금보험료공제\n\d+|"},
            {"target_str": r"IV\. 특별소득공제(소법 §52)\n\d+|"},
            {"target_str": r"V\. 그 밖의 소득공제(조특법)\n\d+|"},
            {"target_str": r"VI\. 세액감면(공제) 및 농어촌특별세\n\d+|"},
            {"target_str": r"I\. 2024년 귀속 연말정산 종합사례\n\d+|"},
            {"target_str": r"II\. 근로소득 원천징수영수증(지급명세서) 작성요령\n\d+|"},
            {"target_str": r"IV\. 수정 원천징수이행상황신고서 작성사례(과다공제)\n\d+|"},
            {"target_str": r"VI\. 홈택스를 이용한 연말정산 신고(근로소득 지급명세서 제출)\n\d+|"},
            {"target_str": r"I\. 사업소득 연말정산\n\d+|"},
            {"target_str": r"II\. 연금소득 연말정산\n\d+|"},
            {"target_str": r"I\. 종교인소득이란?\n\d+|"},
            {"target_str": r"IV\. 종교인소득(기타소득)에 대한 연말정산\n\d+|"},
            {"target_str": r"부록1\. 연말정산 관련 서비스\n\d+|"},
            {"target_str": r"부록2\. 연말정산간소화 서비스\n\d+|"},
            {"target_str": r"부록5\. 연말정산 주요 용어 설명\n\d+|"},
            {"target_str": r"부록6\. 소득·세액공제신고서 첨부서류\n\d+|"},
            {"target_str": r"\bNaN\b"},
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])","replace_str": r"\1\2"},
            {"target_str": r"\s+", "replace_str": r" "},
        ]
    },
    {
        "filename":"연말정산_주택자금·월세액_공제의이해", 
        "page_ranges": [(9, 12), (15, 24), (27, 28), (31, 39), (43, 70)],
        "table_pages_range": "9-12,15-24,27-28,31-39,43-70",
        "replace_string": [
            {"target_str": (r"연말정산 주택자금･월세액 공제의 이해\n\d+|" r"\bNaN\b"),  },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"2024_핵심_개정세법", 
        "page_ranges": [(19, 44), (47, 71), (75, 161)],
        "table_pages_range": "19-44,47-71,75-161",
        "replace_string": [
            {"target_str": (r"2\n0\n2\n5\n\s*달\n라\n지\n는\n\s*세\n금\n제\n도|"  
                r"\n2\n0\n2\n4\n\s*세\n목\n별\n\s*핵\n심\n\s*개\n정\n세\n법|"
                r"\n2\n0\n2\n4\n\s*개\n정\n세\n법\n\s*종\n전\n-\n개\n정\n사\n항\n\s*비\n교\n|"
                r"\s*3\s*❚국민･기업\s*납세자용\s*|"
                r"\s*2\s*0\s*2\s*4\s|"
                r"\s한국세무사회\s|" 
                r"\n7\n❚국민･기업 납세자용|"
                r"\n71\n❚상세본|"),  
            },
            {"target_str": r"([\uAC00-\uD7A3])\n+([\uAC00-\uD7A3])", "replace_str":  r"\1\2" },
            {"target_str": r"\s+", "replace_str": " " },
        ]
    },
    {
        "filename":"주요_공제_항목별_계산사례", 
        "page_ranges": [(1,1), (4, 21)],
        "table_pages_range": "1,4-21",
        "replace_string": [
            {"target_str": r"\bNaN\b"},
            {"target_str": r"\s+", "replace_str": " " },
        ]
    }
    
]

# vector store 에 넣을 데이터 document 모으기 -> all_documents
all_documents = []
# load - 세법 
law_files = get_file_names("data/tax_law")
for file in law_files:
    all_documents.extend(load_process_split_doc_law(file))
# load - 참고자료
for doc in doc_options:
    all_documents.extend(load_process_split_doc(**doc))
    

c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 15 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 37 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 223 of document data/tax_etc/연말정산_신고안내.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:300: UserWarning: Warning: Empty content on page 1 of document data/tax_etc/연말정산_주택자금·월세액_공제의이해.pdf
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\langchain\Lib\site-packages\langchain_community\document_loaders\par

In [3]:
len(all_documents)

6241

In [4]:
# 📌 vector store 생성
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

def set_vector_store(documents):
    embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

    return Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )
    
vector_store = set_vector_store(all_documents)

In [18]:
COLLECTION_NAME = "tax_law"
PERSIST_DIRECTORY = "tax"

# 📌 vector store 불러오기
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
        embedding_function=embedding_model,
        collection_name=COLLECTION_NAME,
        persist_directory=PERSIST_DIRECTORY
    )



In [5]:
vector_store._collection.count()

6241

In [32]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":3 , "fetch_k": 6, "lambda_mult":1}
)

In [45]:
# Prompt Template 생성
messages = [
        ("ai", """
        **기본 원칙**  
        1. 모든 답변은 학습된 세법 데이터 내에서만 작성하세요.  
        - 데이터에 없는 내용은 추측하거나 임의로 생성하지 마세요.  
        - 답변할 정보가 없을 경우, 정직하게 "잘 모르겠습니다."라고 응답하세요.  

        2. 질문을 정확히 이해하고 답변하세요.  
        - 조사(은, 는, 이, 가 등)를 포함한 문장 구조를 바르게 해석하세요.  
        - 질문의 맥락을 파악하고, 필요한 경우 후속 질문을 제안하세요.  

        **답변 작성 가이드**  
        3. 답변의 구성:  
        - **명료하고 간결**하게 작성하며, 비전문가도 이해할 수 있도록 용어를 풀어서 설명하세요.  
        - 질문과 관련된 법 조항, 시행령, 시행규칙의 **번호와 요약된 내용**을 포함하세요.  
        - 법, 시행령, 시행규칙의 계층 관계를 명확히 하세요:  
        - 법률: 과세 원칙 및 제도를 규정.  
        - 시행령: 대통령령으로 법률의 구체적인 사항을 규정.  
        - 시행규칙: 부처에서 정한 세부 절차와 요건.  
        - 필요 시 관련 조항의 전문을 인용하세요.  

        4. 연관 정보 활용:  
        - 질문이 포함된 조항뿐 아니라, 연관된 조항도 참고해 답변의 정확성과 완성도를 높이세요.  
        - 질문이 세부 적용 방법에 관한 것이라면, 시행규칙을 우선 참고하되 법률 및 시행령과의 연관성을 설명하세요.  


        **추가 규칙**  
        5. **법률, 시행령, 시행규칙 구분 응답**  
        - 사용자가 특정 법 조항을 요청할 경우, **법률, 시행령, 시행규칙**을 명확히 구분하여 답변하세요.  
        - 예: "개별소비세법 제1조"를 물어보면 개별소비세법.pdf 파일을 가장 우선으로 참고해서 대해 답변합니다.  
        - 예: "개별소비세법 시행령 제1조"를 물어보면 개별소비세법_시행령.pdf 파일을 가장 우선으로 참고해서 대해 답변합니다.  
        - 예: "개별소비세법 시행규칙 제1조"를 물어보면 개별소비세법_시행규칙.pdf 파일을 가장 우선으로 참고해서 대해 답변합니다.  

        **주의사항**  
        6. 답변 형식:  
        - 모든 답변은 법적 조언이 아닌 **정보 제공 목적**임을 명확히 하세요.  
        - 질문에 해당하지 않는 계층(법, 시행령, 시행규칙)의 정보는 포함하지 마세요.  

        7. 사용자 요청에 따라 세법 구조나 맥락을 간단히 설명해, 이해를 도우세요.

	{context}")"""
        ),
        ("human", "{question}"),
]
prompt_template = ChatPromptTemplate(messages)
# 모델
model = ChatOpenAI(model="gpt-4o")

# output parser
parser = StrOutputParser()

# Chain 구성 retriever(관련문서 조회) -> prompt_template(prompt 생성) -> model(정답) -> output parser
chain = {"context":retriever, "question": RunnablePassthrough()} | prompt_template | model | parser

In [42]:
print(chain.invoke("법인세법 제1조 에 대해 알려줘"))
retriever.invoke("법인세법 제1조 에 대해 알려줘")

법인세법 제1조는 법의 목적에 대해 규정하고 있습니다. 이 조항에서는 법인세법의 목적이 법인세의 부과와 징수에 관한 사항을 규정하여 법인으로부터의 세입을 확보하고, 국가의 재정에 기여하는 것을 목표로 한다고 명시하고 있습니다. 법인세법은 법인에 대한 세금 부과의 기본 원칙과 절차를 제공하며, 이를 통해 공정한 과세를 실현하고자 합니다.

이와 같은 목적 조항은 법인세법의 전반적인 방향성과 의의를 설명하고 있으며, 이후 조항들이 이 목적을 달성하기 위한 구체적인 규정들로 구성되어 있습니다. 

법인세법 제1조에 대한 자세한 내용은 법인세법 전문을 참고하시기 바랍니다.


[Document(metadata={'description': '법인세법 시행규칙 에 관한 문서입니다.', 'keyword': '법인세법 시행규칙', 'source': '법인세법_시행규칙.pdf'}, page_content='법인세법 시행규칙\n \n기획재정부    (국제조세제도과   (외국법인)) 044-215-4652\n기획재정부    (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 시행령 에 관한 문서입니다.', 'keyword': '법인세법 시행령', 'source': '법인세법_시행령.pdf'}, page_content='법인세법 시행령\n \n기획재정부    (국제조세제도과   (외국법인)) 044-215-4652\n기획재정부    (법인세제과) 044-215-4221\n       제1장 총칙'),
 Document(metadata={'description': '법인세법 에 관한 문서입니다.', 'keyword': '법인세법', 'source': '법인세법.pdf'}, page_content='법인세법\n \n기획재정부    (법인세제과) 044-215-4221\n기획재정부    (국제조세제도과   (외국법인)) 044-215-4652\n       제1장 총칙')]

In [43]:
print(chain.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘"))
retriever.invoke("개별소비세법 제1조(과세대상과 세율) 에 대해 알려줘")

개별소비세법 제1조는 과세대상과 세율에 관해 규정하고 있습니다. 주요 내용은 다음과 같습니다:

1. **과세대상**: 개별소비세는 특정한 물품, 특정 장소에 대한 입장행위, 특정 장소에서의 유흥음식행위 및 영업행위에 대해 부과됩니다.

2. **세율**:
   - **물품**: 
     - 투전기, 오락용 사행기구 등은 물품가격의 20%의 세율이 적용됩니다.
     - 보석, 귀금속 제품, 고급 시계 등은 대통령령으로 정한 기준가격을 초과하는 부분에 대해 20%의 세율이 적용됩니다.
     - 자동차는 배기량에 따라 5%의 세율이 적용됩니다.
     - 유류 및 가스 등은 수량에 따라 각각 다른 세율이 적용됩니다.
   - **입장행위**: 경마장, 경륜장, 골프장 등 특정 장소의 입장에 대해 정해진 금액의 세율이 부과됩니다.
   - **유흥음식행위**: 유흥주점이나 외국인 전용 유흥음식점 등에서 유흥음식요금의 10%가 과세됩니다.
   - **영업행위**: 카지노 등에서의 영업행위는 연간 총매출액에 따라 과세됩니다.

이러한 규정들은 국민경제의 효율적 운용을 위해 필요한 경우 대통령령으로 세율을 조정할 수 있습니다. 과세물품의 판정은 물품의 형태, 용도, 성질 등의 특성에 따라 이루어집니다.


[Document(metadata={'description': '개별소비세법 시행령 제1조에 관한 문서입니다.', 'keyword': '개별소비세법 시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='제1조    (과세물품ㆍ과세장소 및 과세유흥장소의 세목등) 「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n   (「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.'),
 Document(metadata={'description': '개별소비세법 제1조에 관한 문서입니다.', 'keyword': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='제1조    (과세대상과 세율) ① 개별소비세는 특정한 물품, 특정한 장소 입장행위   (入場行爲), 특정한 장소에서의 유흥음식행\n위   (遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다. \n② 개별소비세를 부과할 물품   (이하 “과세물품”이라 한다)과 그 세율은 다음과 같다.\n1. 다음 각 목의 물품에 대해서는 그 물품가격에 100분의 20의 세율을 적용한다.\n가. 투전기   (投錢機), 오락용 사행기구   (射倖器具), 그 밖의 오락용품\n나. 수렵용 총포류\n2. 다음 각 목의 물품에 대해서는 그 물품가격 중 대통령령으로 정하는 기준가격   (이하 “기준가격”이라 한다)을 초과\n하는 부분의 가격   (이하 이 호에서 “과세가격”이라 한다)에 해당 세율을 적용한다.\n가. 다음의 물품에 대해서는 과세가격의 100분의 20\n1) 보석, 진주, 별갑   (鼈甲), 산호, 호\n박   (琥珀) 및 상아와 이를 사용한 제품   (나석을 사용한 제품은

In [36]:
print(chain.invoke("개별소비세법 과세대상과 세율 에 대해 알려줘"))
retriever.invoke("개별소비세법 과세대상과 세율 에 대해 알려줘")

개별소비세법의 과세대상과 세율에 대해 설명드리겠습니다.

**과세대상:**
개별소비세법은 특정한 소비재나 서비스에 대해 과세하는 세금입니다. 주로 사치품, 특정한 고급 소비재, 그리고 환경에 영향을 미치는 제품 등이 과세대상에 포함됩니다. 과세대상은 법률과 시행령에 따라 구체적으로 규정됩니다. 예를 들어, 고급 자동차, 보석류, 고급 가전제품 등이 그 대상에 포함될 수 있습니다.

**세율:**
개별소비세의 세율은 과세대상 품목에 따라 다르게 적용됩니다. 일반적으로 고급 소비재나 사치품에 대해 높은 세율이 부과됩니다. 세율은 법률에서 기본적인 틀을 제공하고, 시행령과 시행규칙에서 구체적으로 명시됩니다. 따라서 각 품목에 대한 구체적인 세율은 관련 법령을 참조해야 합니다.

개별소비세법의 조항을 참고해 보다 상세한 정보를 원하시면, 해당 법령의 구체적인 조항을 확인하는 것이 필요합니다. 법령의 변화가 있을 수 있으므로, 최신 정보를 확인하는 것도 중요합니다. 필요하신 경우, 특정 품목에 대한 세율이나 과세 기준을 알려주시면 보다 구체적으로 안내드릴 수 있습니다.


[Document(metadata={'description': '개별소비세법 시행규칙 에 관한 문서입니다.', 'keyword': '개별소비세법 시행규칙', 'source': '개별소비세법_시행규칙.pdf'}, page_content='개별소비세법 시행규칙\n \n기획재정부    (환경에너지세제과) 044-215-4331, 4336\n기획재정부    (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 에 관한 문서입니다.', 'keyword': '개별소비세법', 'source': '개별소비세법.pdf'}, page_content='개별소비세법\n \n기획재정부    (환경에너지세제과) 044-215-4331, 4336\n기획재정부    (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 시행령 에 관한 문서입니다.', 'keyword': '개별소비세법 시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='개별소비세법 시행령\n \n기획재정부    (환경에너지세제과) 044-215-4331, 4336\n기획재정부    (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336')]

In [ ]:
print(chain.invoke("개별소비세법 제1조 에 대해 알려줘"))
retriever.invoke("개별소비세법 제1조 에 대해 알려줘")

죄송하지만, 개별소비세법 제1조에 대한 구체적인 내용은 현재 제공된 데이터에는 포함되어 있지 않습니다. 다만 일반적으로 법률 제1조는 해당 법의 목적을 규정하는 경우가 많습니다. 개별소비세법 제1조의 구체적인 내용은 법령 정보를 제공하는 공식 웹사이트나 관련 법률 자료를 통해 확인하는 것이 좋습니다. 추가적인 도움이 필요하시면 말씀해 주세요.


[Document(metadata={'description': '개별소비세법 시행규칙 에 관한 문서입니다.', 'keyword': '개별소비세법 시행규칙', 'source': '개별소비세법_시행규칙.pdf'}, page_content='개별소비세법 시행규칙\n \n기획재정부    (환경에너지세제과) 044-215-4331, 4336\n기획재정부    (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336'),
 Document(metadata={'description': '개별소비세법 시행령 제1조에 관한 문서입니다.', 'keyword': '개별소비세법 시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='제1조    (과세물품ㆍ과세장소 및 과세유흥장소의 세목등) 「개별소비세법」 제1조제6항에 따른 과세물품의 세목은 별표\n1과 같이 하고, 과세장소의 종류는 별표 2와 같이 하며, 과세유흥장소의 종류는 유흥주점ㆍ외국인전용 유흥음식점\n및 그 밖에 이와 유사한 장소로 하고, 과세영업장소의 종류는 「관광진흥법」 제5조제1항에 따라 허가를 받은 카지노\n   (「폐광지역개발 지원에 관한 특별법」 제11조에 따라 허가를 받은 카지노를 포함한다)로 한다.'),
 Document(metadata={'description': '개별소비세법 시행령 에 관한 문서입니다.', 'keyword': '개별소비세법 시행령', 'source': '개별소비세법_시행령.pdf'}, page_content='개별소비세법 시행령\n \n기획재정부    (환경에너지세제과) 044-215-4331, 4336\n기획재정부    (환경에너지세제과 - 자동차 부분) 044-215-4333, 4336')]

## 평가

In [ ]:
from ragas import EvaluationDataset, RunConfig, evaluate
from datasets import Dataset
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

from datasets import Dataset

# LangChain 모델 래핑
langchain_model = LangchainLLMWrapper(model)

# 테스트 데이터 준비 (예시)
test_data = [
    # {
    #     "question": "개별소비세법의 목적은 무엇인가요?",
    #     "answer": chain.invoke("개별소비세법의 목적은 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")],
    #     "ground_truths": ["개별소비세법의 목적은 특정 물품과 특정 장소에 대한 소비세를 부과하여 국가 재정수입을 확보하고 소비를 조정하는 것입니다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법의 목적은 무엇인가요?")])
    # },
    # {
    #     "question": "",
    #     "answer": chain.invoke("개별소비세법 제1조(과세대상과 세율)가 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조가 무엇인가요?")],
    #     "ground_truths": ["제1조(과세대상과 세율) ① 개별소비세는 특정한 물품, 특정한 장소 입장행위(入場行爲), 특정한 장소에서의 유흥음식행위(遊興飮食行爲) 및 특정한 장소에서의 영업행위에 대하여 부과한다. <개정 2010. 1. 1.>② 개별소비세를 부과할 물품(이하 “과세물품”이라 한다)과 그 세율은 다음과 같다.",],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("개별소비세법 제1조(과세대상과 세율)가 무엇인가요?")])
    # },
    # {
    #     "question": "조세범 처벌절차법의 목적이 무엇인가요?",
    #     "answer": chain.invoke("조세범 처벌절차법의 목적이 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")],
    #     "ground_truths": ["제1조(목적) 이 법은 조세범칙사건(犯則事件)을 공정하고 효율적으로 처리하기 위하여 조세범칙사건의 조사 및 그 처분에 관한 사항을 정함을 목적으로 한다"],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("조세범 처벌절차법의 목적이 무엇인가요?")])
    # },
    # {
    #     "question": "법인세 제1조가 무엇인가요?",
    #     "answer": chain.invoke("법인세 제1조가 무엇인가요?"),
    #     "contexts": [doc.page_content for doc in retriever.get_relevant_documents("법인세 제1조가 무엇인가요?")],
    #     "ground_truths": ["이 법은 법인세의 과세 요건과 절차를 규정함으로써 법인세를 공정하게 과세하고, 납세의무의 적절한 이행을 확보하며, 재정수입의 원활한 조달에 이바지함을 목적으로 한다."],
    #     "reference": "\n".join([doc.page_content for doc in retriever.get_relevant_documents("법인세 제1조가 무엇인가요?")])
    # }
    
]

# Dataset 생성
dataset = Dataset.from_list(test_data)

# 평가 실행
result = evaluate(
    dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ],
    llm=langchain_model,
)

# 결과 출력
print(result)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'faithfulness': 0.2917, 'answer_relevancy': 0.8254, 'context_precision': 1.0000, 'context_recall': 0.1034}


In [48]:
# from ragas import EvaluationDataset, RunConfig, evaluate
# from ragas.metrics import (
#     LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
# )
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

## GPT-4o 모델을 사용하여 평가 
model_name = "gpt-4o"
model = ChatOpenAI(model=model_name)
langchain_model = LangchainLLMWrapper(model)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextRecall(llm=langchain_model),
    LLMContextPrecisionWithReference(llm=langchain_model),
    Faithfulness(llm=langchain_model),
    AnswerRelevancy(llm=langchain_model, embeddings=eval_embedding)
]
result = evaluate(dataset=dataset, metrics=metrics)

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

In [49]:
result

{'context_recall': 1.0000, 'llm_context_precision_with_reference': 0.5000, 'faithfulness': 0.0000, 'answer_relevancy': 0.0000}

In [ ]:
# DataFrame 생성
df_result = pd.DataFrame(test_data)

# 최대 열 너비 설정
pd.set_option('display.max_colwidth', None)

# 최대 행 수 설정
pd.set_option('display.max_rows', None)

df_result